# W_GL_SEGMENT_D ETL Process
### SCD Type 2 Dimension Load - GL Segment Dimension
**Package:** SILOS_SIL_GLSEGMENTDIMENSION

In [0]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL,
  999 AS DATASOURCE_NUM_ID,
  '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID,
  2625027 AS ETL_PROC_WID,
  12345 AS EXECUTION_ID,
  30 AS PRUNE_DAYS,
  '1970-01-01 00:00:00' AS LOW_DATE,
  '3714-01-01 00:00:00' AS HI_DT
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLSEGMENTDIMENSION'
  AND DATASOURCE_NUM_ID = 999
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1';

In [0]:
%sql
-- 2) STAGING: Prepare source data from W_GL_SEGMENT_DS with aggregations and lookups
CREATE OR REPLACE TEMP VIEW stg_gl_segment_source AS
SELECT
  ds.DATASOURCE_NUM_ID,
  ds.INTEGRATION_ID,
  MAX(ds.SEGMENT_LOV_ID) AS SEGMENT_LOV_ID,
  MAX(ds.SEGMENT_VAL_CODE) AS SEGMENT_VAL_CODE,
  MAX(ds.CREATED_BY_ID) AS CREATED_BY_ID,
  MAX(ds.CHANGED_BY_ID) AS CHANGED_BY_ID,
  MAX(ds.CREATED_ON_DT) AS CREATED_ON_DT,
  MAX(ds.CHANGED_ON_DT) AS CHANGED_ON_DT,
  MAX(ds.AUX1_CHANGED_ON_DT) AS AUX1_CHANGED_ON_DT,
  MAX(ds.AUX2_CHANGED_ON_DT) AS AUX2_CHANGED_ON_DT,
  MAX(ds.AUX3_CHANGED_ON_DT) AS AUX3_CHANGED_ON_DT,
  MAX(ds.AUX4_CHANGED_ON_DT) AS AUX4_CHANGED_ON_DT,
  ds.SRC_EFF_FROM_DT,
  ds.SRC_EFF_TO_DT,
  MAX(ds.DELETE_FLG) AS DELETE_FLG,
  MAX(ds.SET_ID) AS SET_ID,
  MAX(ds.TENANT_ID) AS TENANT_ID,
  MAX(ds.X_CUSTOM) AS X_CUSTOM
FROM workspace.oracle_edw.w_gl_segment_ds ds
WHERE ds.DATASOURCE_NUM_ID = 999
GROUP BY
  ds.DATASOURCE_NUM_ID,
  ds.INTEGRATION_ID,
  ds.SRC_EFF_FROM_DT,
  ds.SRC_EFF_TO_DT;

In [0]:
%sql
-- 3) ENRICHED STAGING: Join with user lookups and prepare for SCD processing
CREATE OR REPLACE TEMP VIEW stg_gl_segment_enriched AS
SELECT
  0 AS SCD1_WID,
  src.SEGMENT_LOV_ID,
  COALESCE(src.SEGMENT_VAL_CODE, '__NOT_APPLICABLE__') AS SEGMENT_VAL_CODE,
  COALESCE(created_by.ROW_WID, 0) AS CREATED_BY_WID,
  COALESCE(changed_by.ROW_WID, 0) AS CHANGED_BY_WID,
  src.CREATED_ON_DT,
  src.CHANGED_ON_DT,
  src.AUX1_CHANGED_ON_DT,
  src.AUX2_CHANGED_ON_DT,
  src.AUX3_CHANGED_ON_DT,
  src.AUX4_CHANGED_ON_DT,
  src.SRC_EFF_FROM_DT,
  src.SRC_EFF_TO_DT,
  CASE WHEN src.DELETE_FLG = 'Y' THEN 'Y' ELSE 'N' END AS DELETE_FLG,
  src.DATASOURCE_NUM_ID,
  src.INTEGRATION_ID,
  src.SET_ID,
  src.TENANT_ID,
  src.X_CUSTOM,
  COALESCE(src.SRC_EFF_FROM_DT, to_timestamp('1970-01-01 00:00:00', 'yyyy-MM-dd HH:mm:ss')) AS EFFECTIVE_FROM_DT,
  to_timestamp('3714-01-01 00:00:00', 'yyyy-MM-dd HH:mm:ss') AS EFFECTIVE_TO_DT,
  'Y' AS IND_CURRENT_FLG,
  'I' AS IND_UPDATE
FROM stg_gl_segment_source src
LEFT JOIN workspace.oracle_edw.w_user_d created_by
  ON created_by.INTEGRATION_ID = src.CREATED_BY_ID
  AND created_by.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND created_by.CURRENT_FLG = 'Y'
LEFT JOIN workspace.oracle_edw.w_user_d changed_by
  ON changed_by.INTEGRATION_ID = src.CHANGED_BY_ID
  AND changed_by.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND changed_by.CURRENT_FLG = 'Y';

In [0]:
%sql
-- 4) FLOW TABLE: Identify new, changed, and unchanged records (SCD Type 2 logic)
CREATE OR REPLACE TEMP VIEW i_flow_table AS
SELECT
  stg.SCD1_WID,
  stg.SEGMENT_LOV_ID,
  stg.SEGMENT_VAL_CODE,
  stg.CREATED_BY_WID,
  stg.CHANGED_BY_WID,
  stg.CREATED_ON_DT,
  stg.CHANGED_ON_DT,
  stg.AUX1_CHANGED_ON_DT,
  stg.AUX2_CHANGED_ON_DT,
  stg.AUX3_CHANGED_ON_DT,
  stg.AUX4_CHANGED_ON_DT,
  stg.SRC_EFF_FROM_DT,
  stg.SRC_EFF_TO_DT,
  stg.DELETE_FLG,
  stg.DATASOURCE_NUM_ID,
  stg.INTEGRATION_ID,
  stg.SET_ID,
  stg.TENANT_ID,
  stg.X_CUSTOM,
  stg.EFFECTIVE_FROM_DT,
  stg.EFFECTIVE_TO_DT,
  stg.IND_CURRENT_FLG,
  CASE
    -- New record (INSERT)
    WHEN tgt.INTEGRATION_ID IS NULL THEN 'I'
    -- Changed record that requires update to existing row (UPDATE)
    WHEN tgt.INTEGRATION_ID IS NOT NULL 
      AND tgt.EFFECTIVE_FROM_DT = stg.EFFECTIVE_FROM_DT THEN 'U'
    -- New version of existing record (SCD INSERT)
    WHEN tgt.INTEGRATION_ID IS NOT NULL 
      AND tgt.EFFECTIVE_FROM_DT < stg.EFFECTIVE_FROM_DT THEN 'S'
    ELSE 'I'
  END AS IND_UPDATE
FROM stg_gl_segment_enriched stg
LEFT JOIN workspace.oracle_edw.w_gl_segment_d tgt
  ON tgt.DATASOURCE_NUM_ID = stg.DATASOURCE_NUM_ID
  AND tgt.INTEGRATION_ID = stg.INTEGRATION_ID
  AND tgt.CURRENT_FLG = 'Y'
WHERE stg.DELETE_FLG = 'N';

In [0]:
%sql
-- 5) SOFT DELETE QUEUE: Identify records to be soft deleted
CREATE OR REPLACE TEMP VIEW delete_queue AS
SELECT
  ds.SRC_EFF_FROM_DT,
  ds.DATASOURCE_NUM_ID,
  ds.INTEGRATION_ID
FROM workspace.oracle_edw.w_gl_segment_ds ds
WHERE ds.DELETE_FLG = 'Y'
  AND ds.DATASOURCE_NUM_ID = 999;

In [0]:
%sql
-- 6) CLOSE FUTURE-DATED RECORDS: Update EFFECTIVE_TO_DT for records that will be superseded
MERGE INTO workspace.oracle_edw.w_gl_segment_d AS tgt
USING (
  SELECT DISTINCT
    t.ROW_WID,
    t.DATASOURCE_NUM_ID,
    t.INTEGRATION_ID,
    t.EFFECTIVE_FROM_DT,
    i.EFFECTIVE_FROM_DT AS new_effective_to_dt
  FROM workspace.oracle_edw.w_gl_segment_d t
  INNER JOIN i_flow_table i
    ON t.DATASOURCE_NUM_ID = i.DATASOURCE_NUM_ID
    AND t.INTEGRATION_ID = i.INTEGRATION_ID
    AND t.EFFECTIVE_FROM_DT < i.EFFECTIVE_FROM_DT
    AND i.DELETE_FLG = 'N'
    AND i.IND_UPDATE IN ('I', 'S')
  WHERE t.DELETE_FLG = 'N'
) src
ON tgt.ROW_WID = src.ROW_WID
WHEN MATCHED THEN
  UPDATE SET
    tgt.EFFECTIVE_TO_DT = src.new_effective_to_dt,
    tgt.CURRENT_FLG = 'N',
    tgt.W_UPDATE_DT = current_timestamp(),
    tgt.ETL_PROC_WID = 2625027;

In [0]:
%sql
-- 7) SOFT DELETE: Mark records as deleted based on delete queue
MERGE INTO workspace.oracle_edw.w_gl_segment_d AS tgt
USING delete_queue AS src
ON tgt.SRC_EFF_FROM_DT = src.SRC_EFF_FROM_DT
  AND tgt.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND tgt.INTEGRATION_ID = src.INTEGRATION_ID
WHEN MATCHED THEN
  UPDATE SET
    tgt.DELETE_FLG = 'Y',
    tgt.CURRENT_FLG = 'N',
    tgt.W_UPDATE_DT = current_timestamp(),
    tgt.ETL_PROC_WID = 2625027;

In [0]:
%sql
-- 8) INSERT NEW DIMENSION RECORDS: Add new records and new versions (SCD Type 2)
INSERT INTO workspace.oracle_edw.w_gl_segment_d
(
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  SEGMENT_LOV_ID,
  SEGMENT_VAL_CODE,
  SET_ID,
  TENANT_ID,
  X_CUSTOM,
  CREATED_BY_WID,
  CHANGED_BY_WID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  AUX4_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  SRC_EFF_TO_DT,
  DELETE_FLG,
  W_INSERT_DT,
  W_UPDATE_DT,
  ETL_PROC_WID,
  SCD1_WID,
  CURRENT_FLG,
  EFFECTIVE_FROM_DT,
  EFFECTIVE_TO_DT
)
SELECT
  i.DATASOURCE_NUM_ID,
  i.INTEGRATION_ID,
  i.SEGMENT_LOV_ID,
  i.SEGMENT_VAL_CODE,
  i.SET_ID,
  i.TENANT_ID,
  i.X_CUSTOM,
  i.CREATED_BY_WID,
  i.CHANGED_BY_WID,
  i.CREATED_ON_DT,
  i.CHANGED_ON_DT,
  i.AUX1_CHANGED_ON_DT,
  i.AUX2_CHANGED_ON_DT,
  i.AUX3_CHANGED_ON_DT,
  i.AUX4_CHANGED_ON_DT,
  i.SRC_EFF_FROM_DT,
  i.SRC_EFF_TO_DT,
  i.DELETE_FLG,
  current_timestamp() AS W_INSERT_DT,
  current_timestamp() AS W_UPDATE_DT,
  2625027 AS ETL_PROC_WID,
  scd1.SCD1_WID,
  i.IND_CURRENT_FLG AS CURRENT_FLG,
  i.EFFECTIVE_FROM_DT,
  i.EFFECTIVE_TO_DT
FROM i_flow_table i
INNER JOIN workspace.oracle_edw.w_gl_segment_t1_d scd1
  ON scd1.DATASOURCE_NUM_ID = i.DATASOURCE_NUM_ID
  AND scd1.INTEGRATION_ID = i.INTEGRATION_ID
WHERE i.IND_UPDATE IN ('I', 'S');

In [0]:
%sql
-- 9) UPDATE EXISTING ROWS: Update Type 1 columns on current dimension records
MERGE INTO workspace.oracle_edw.w_gl_segment_d AS tgt
USING (
  SELECT
    EFFECTIVE_FROM_DT,
    DATASOURCE_NUM_ID,
    INTEGRATION_ID,
    SEGMENT_LOV_ID,
    SEGMENT_VAL_CODE,
    CREATED_BY_WID,
    CHANGED_BY_WID,
    CREATED_ON_DT,
    CHANGED_ON_DT,
    AUX1_CHANGED_ON_DT,
    AUX2_CHANGED_ON_DT,
    AUX3_CHANGED_ON_DT,
    AUX4_CHANGED_ON_DT,
    SRC_EFF_FROM_DT,
    SRC_EFF_TO_DT,
    DELETE_FLG,
    SET_ID,
    TENANT_ID,
    X_CUSTOM
  FROM i_flow_table
  WHERE IND_UPDATE = 'U'
) src
ON tgt.EFFECTIVE_FROM_DT = src.EFFECTIVE_FROM_DT
  AND tgt.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND tgt.INTEGRATION_ID = src.INTEGRATION_ID
WHEN MATCHED THEN
  UPDATE SET
    tgt.SEGMENT_LOV_ID = src.SEGMENT_LOV_ID,
    tgt.SEGMENT_VAL_CODE = src.SEGMENT_VAL_CODE,
    tgt.CREATED_BY_WID = src.CREATED_BY_WID,
    tgt.CHANGED_BY_WID = src.CHANGED_BY_WID,
    tgt.CREATED_ON_DT = src.CREATED_ON_DT,
    tgt.CHANGED_ON_DT = src.CHANGED_ON_DT,
    tgt.AUX1_CHANGED_ON_DT = src.AUX1_CHANGED_ON_DT,
    tgt.AUX2_CHANGED_ON_DT = src.AUX2_CHANGED_ON_DT,
    tgt.AUX3_CHANGED_ON_DT = src.AUX3_CHANGED_ON_DT,
    tgt.AUX4_CHANGED_ON_DT = src.AUX4_CHANGED_ON_DT,
    tgt.SRC_EFF_FROM_DT = src.SRC_EFF_FROM_DT,
    tgt.SRC_EFF_TO_DT = src.SRC_EFF_TO_DT,
    tgt.DELETE_FLG = src.DELETE_FLG,
    tgt.SET_ID = src.SET_ID,
    tgt.TENANT_ID = src.TENANT_ID,
    tgt.X_CUSTOM = src.X_CUSTOM,
    tgt.W_UPDATE_DT = current_timestamp(),
    tgt.ETL_PROC_WID = 2625027;

In [0]:
%sql
-- 10) HISTORIZE OLD ROWS: Close out old records when new versions are inserted
MERGE INTO workspace.oracle_edw.w_gl_segment_d AS tgt
USING (
  SELECT DISTINCT
    t.ROW_WID,
    MIN(i.EFFECTIVE_FROM_DT) AS new_effective_to_dt
  FROM workspace.oracle_edw.w_gl_segment_d t
  INNER JOIN i_flow_table i
    ON i.DATASOURCE_NUM_ID = t.DATASOURCE_NUM_ID
    AND i.INTEGRATION_ID = t.INTEGRATION_ID
    AND i.EFFECTIVE_FROM_DT > t.EFFECTIVE_FROM_DT
    AND i.EFFECTIVE_FROM_DT < t.EFFECTIVE_TO_DT
    AND i.IND_UPDATE IN ('I', 'S')
  WHERE t.DELETE_FLG = 'N'
  GROUP BY t.ROW_WID
) src
ON tgt.ROW_WID = src.ROW_WID
WHEN MATCHED THEN
  UPDATE SET
    tgt.CURRENT_FLG = 'N',
    tgt.EFFECTIVE_TO_DT = src.new_effective_to_dt,
    tgt.W_UPDATE_DT = current_timestamp(),
    tgt.ETL_PROC_WID = 2625027;

In [0]:
%sql
-- 11) UPDATE CONTROL TABLE: Update W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT
    p.DATASOURCE_NUM_ID AS datasource_num_id,
    'SILOS_SIL_GLSEGMENTDIMENSION' AS package_name,
    'W_GL_SEGMENT_D' AS target_table_name,
    p.ETL_USAGE_CODE AS etl_usage_code,
    p.ETL_PROC_WID AS etl_proc_wid,
    p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.package_name = src.package_name
  AND tgt.etl_usage_code = src.etl_usage_code
WHEN MATCHED THEN
  UPDATE SET
    tgt.target_table_name = src.target_table_name,
    tgt.etl_proc_wid = src.etl_proc_wid,
    tgt.load_plan_id = src.load_plan_id,
    tgt.wip_load_start_date = src.wip_load_start_date,
    tgt.etl_load_date = src.etl_load_date,
    tgt.committed = src.committed
WHEN NOT MATCHED THEN
  INSERT (
    datasource_num_id, package_name, target_table_name, etl_usage_code,
    etl_proc_wid, load_plan_id, wip_load_start_date, last_max_date, etl_load_date, committed
  )
  VALUES (
    src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
    src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed
  );

In [0]:
%sql
-- 12) INSERT HISTORY LOG: Record ETL execution in history table
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  p.DATASOURCE_NUM_ID,
  'SILOS_SIL_GLSEGMENTDIMENSION',
  'W_GL_SEGMENT_D',
  p.ETL_USAGE_CODE,
  p.ETL_PROC_WID,
  p.EXECUTION_ID,
  p.ETL_PROC_WID AS SESSION_ID,
  date_add(current_timestamp(), -p.PRUNE_DAYS) AS WIP_LOAD_START_DATE,
  NULL AS LAST_MAX_DATE,
  current_timestamp() AS ETL_LOAD_DATE,
  CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS COMMITTED
FROM etl_params p;

In [0]:
%sql
-- 13) CLEANUP: Drop temporary views
DROP VIEW IF EXISTS i_flow_table;
DROP VIEW IF EXISTS delete_queue;
DROP VIEW IF EXISTS stg_gl_segment_enriched;
DROP VIEW IF EXISTS stg_gl_segment_source;
DROP VIEW IF EXISTS etl_params;